In [1]:
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn.functional as F
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
import torch_geometric.transforms as T
from torch_geometric.data import DataLoader
from torch_geometric.nn import SAGEConv

In [2]:
dataset = PygNodePropPredDataset(name='ogbn-arxiv', 
                                 root='../data/dataset/',
                                 transform=T.ToSparseTensor())

In [3]:
torch.cuda.is_available()

True

In [4]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

In [5]:
data = dataset[0]

In [6]:
data

Data(adj_t=[169343, 169343, nnz=1166243], node_year=[169343, 1], x=[169343, 128], y=[169343, 1])

In [7]:
data = data.to(device)

In [8]:
split_idx = dataset.get_idx_split()
train_idx = split_idx['train'].to(device)

In [9]:
evaluator = Evaluator(name='ogbn-arxiv')

In [10]:
data['x'].shape[1]

128

In [11]:
data['y'][:, 0].unique().shape[0]

40

In [12]:
n_iters = 3000
epochs = 2000
log_steps = 100
input_dim = data['x'].shape[1]
hidden_channels = input_dim * 2
output_dim = data['y'][:, 0].unique().shape[0]
lr_rate = 0.001

In [13]:
class OneLayer(torch.nn.Module):
    def __init__(self, input_dim, hidden_channels, output_dim):
        super(OneLayer, self).__init__()
        
        self.convs = torch.nn.ModuleList()
        self.bns = torch.nn.ModuleList()
        
        self.convs.append(SAGEConv(input_dim, hidden_channels))
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        
        self.convs.append(SAGEConv(hidden_channels, output_dim))
        

    def forward(self, x, adj_t):
        x = self.convs[0](x, adj_t)
        x = self.bns[0](x)
        x = self.convs[1](x, adj_t)
        return x.log_softmax(dim=-1)

In [14]:
model = OneLayer(input_dim, hidden_channels, output_dim).to(device)

In [15]:
def train(model, data, train_idx, optimizer):
    model.train()

    optimizer.zero_grad()
    out = model(data.x, data.adj_t)[train_idx]
    loss = F.nll_loss(out, data.y.squeeze(1)[train_idx])
    loss.backward()
    optimizer.step()

    return loss.item()

In [16]:
@torch.no_grad()
def test(model, data, split_idx, evaluator):
    model.eval()

    out = model(data.x, data.adj_t)
    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']

    return train_acc, valid_acc, test_acc

In [17]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr_rate)
for epoch in range(1, 1 + epochs):
    loss = train(model, data, train_idx, optimizer)
    result = test(model, data, split_idx, evaluator)

    if epoch % log_steps == 0:
        train_acc, valid_acc, test_acc = result
        print(f'Epoch: {epoch:02d}, '
              f'Loss: {loss:.4f}, '
              f'Train: {100 * train_acc:.2f}%, '
              f'Valid: {100 * valid_acc:.2f}% '
              f'Test: {100 * test_acc:.2f}%')

Epoch: 100, Loss: 1.3086, Train: 62.62%, Valid: 59.35% Test: 52.61%
Epoch: 200, Loss: 1.2127, Train: 64.24%, Valid: 60.21% Test: 53.84%
Epoch: 300, Loss: 1.1820, Train: 64.78%, Valid: 60.12% Test: 53.63%
Epoch: 400, Loss: 1.1701, Train: 65.06%, Valid: 59.94% Test: 53.59%
Epoch: 500, Loss: 1.1643, Train: 65.29%, Valid: 59.60% Test: 53.23%
Epoch: 600, Loss: 1.1615, Train: 65.41%, Valid: 59.57% Test: 53.15%
Epoch: 700, Loss: 1.1599, Train: 65.47%, Valid: 59.60% Test: 53.17%
Epoch: 800, Loss: 1.1589, Train: 65.45%, Valid: 59.39% Test: 52.91%
Epoch: 900, Loss: 1.1583, Train: 65.50%, Valid: 59.43% Test: 53.02%
Epoch: 1000, Loss: 1.1580, Train: 65.48%, Valid: 59.49% Test: 53.12%
Epoch: 1100, Loss: 1.1577, Train: 65.49%, Valid: 59.43% Test: 52.92%
Epoch: 1200, Loss: 1.1575, Train: 65.52%, Valid: 59.42% Test: 53.00%
Epoch: 1300, Loss: 1.1574, Train: 65.51%, Valid: 59.35% Test: 52.90%
Epoch: 1400, Loss: 1.1573, Train: 65.53%, Valid: 59.32% Test: 52.93%
Epoch: 1500, Loss: 1.1572, Train: 65.53%, V